In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableLambda,RunnableParallel,RunnablePassthrough
from langchain_core.prompts import PromptTemplate

In [2]:
loader=PyPDFLoader(r"C:\Users\yash\Downloads\Introduction_to_Data_and_Data_Science.pdf")

In [3]:
pdf=loader.load()

In [4]:
splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200
)

In [17]:
chunk=splitter.split_documents(pdf)

In [19]:
for i in range(len(chunk)):
    chunk[i].page_content=' '.join(chunk[i].page_content.split())

In [20]:
chunk[2]

Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-11-09T10:16:34+02:00', 'author': 'Hristina  Hristova', 'moddate': '2023-11-09T10:16:34+02:00', 'source': 'C:\\Users\\yash\\Downloads\\Introduction_to_Data_and_Data_Science.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='various types. Instead of tackling the entire dataset and running the risk of becoming overwhelmed, you separate it into easier to digest chunks and study them individually and examine how they relate to other parts. And that’s analysis in a nutshell. One important thing to remember, however, is that you perform analyses on things that have already happened in the past. Such as using an analysis to explain how a story ended the way it did or how there was a decrease')

In [21]:
from langchain_community.embeddings import OllamaEmbeddings

In [22]:
embedding=OllamaEmbeddings(model='nomic-embed-text')

In [23]:
vectorstore=FAISS.from_documents(chunk,embedding)

In [24]:
retriver=vectorstore.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [25]:
retriver.invoke('what is tools')

[Document(id='6d20175d-0f70-4a5f-acf4-b2d5a9d7d627', metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-11-09T10:16:34+02:00', 'author': 'Hristina  Hristova', 'moddate': '2023-11-09T10:16:34+02:00', 'source': 'C:\\Users\\yash\\Downloads\\Introduction_to_Data_and_Data_Science.pdf', 'total_pages': 6, 'page': 2, 'page_label': '3'}, page_content='You can basically split the relevant tools into two categories—programming languages and software. Knowing a programming language enables you to devise programs that can execute specific operations. Moreover, you can reuse these programs whenever you need to execute the same action. As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated'),
 Document(id='0a963162-3475-4a55-91e7-8ae9ceab08a6', metadata={'producer': 'Microsoft® Word for Microsoft 365', 

In [26]:
retrive_docs=retriver.invoke('what is tools')

In [27]:
retrive_docs

[Document(id='6d20175d-0f70-4a5f-acf4-b2d5a9d7d627', metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-11-09T10:16:34+02:00', 'author': 'Hristina  Hristova', 'moddate': '2023-11-09T10:16:34+02:00', 'source': 'C:\\Users\\yash\\Downloads\\Introduction_to_Data_and_Data_Science.pdf', 'total_pages': 6, 'page': 2, 'page_label': '3'}, page_content='You can basically split the relevant tools into two categories—programming languages and software. Knowing a programming language enables you to devise programs that can execute specific operations. Moreover, you can reuse these programs whenever you need to execute the same action. As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated'),
 Document(id='0a963162-3475-4a55-91e7-8ae9ceab08a6', metadata={'producer': 'Microsoft® Word for Microsoft 365', 

In [28]:
context_text='\n\n'.join(doc.page_content for doc in retrive_docs)

In [30]:
prompt=PromptTemplate(template='''
You are a assistant bot to help to retrive information from this pdf.
You will ONLY ANSWER the question from the given transcript.
If you dont know the answer just say you dont know
Give answer from the following context
{context}
Question:{question}'''
                      
)

In [31]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are a assistant bot to help to retrive information from this pdf.\nYou will ONLY ANSWER the question from the given transcript.\nIf you dont know the answer just say you dont know\nGive answer from the following context\n{context}\nQuestion:{question}')

In [33]:
question='what is data science'


In [36]:
final_promt=prompt.invoke({'context':context_text,'question':question})

In [37]:
final_promt

StringPromptValue(text='\nYou are a assistant bot to help to retrive information from this pdf.\nYou will ONLY ANSWER the question from the given transcript.\nIf you dont know the answer just say you dont know\nGive answer from the following context\nYou can basically split the relevant tools into two categories—programming languages and software. Knowing a programming language enables you to devise programs that can execute specific operations. Moreover, you can reuse these programs whenever you need to execute the same action. As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated\n\nare constantly applied. As a final note, remember the following. Should you have the relevant business and theoretical knowledge, learning a software tool is relatively easy as opposed to learning a programming language. More importantly, it will be sufficient for your need to create

In [38]:
from langchain_ollama.chat_models import ChatOllama


In [39]:
chat=ChatOllama(model='llama3.1:8b')

In [40]:
answer=chat.invoke(final_promt)

In [42]:
answer

AIMessage(content="I don't know", additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-12-23T09:52:15.96652Z', 'done': True, 'done_reason': 'stop', 'total_duration': 18821789900, 'load_duration': 5275938700, 'prompt_eval_count': 277, 'prompt_eval_duration': 12656663300, 'eval_count': 5, 'eval_duration': 874611600, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b4a9f-5ed2-7483-907f-129360eee7b6-0', usage_metadata={'input_tokens': 277, 'output_tokens': 5, 'total_tokens': 282})

In [43]:
def docs_imfo(retriver_docs):
    context_text='/n/n'.join(doc.page_content for doc in retrive_docs)

In [44]:
parallel_chain=RunnableParallel({'context':retriver|RunnableLambda(docs_imfo),
                                'question':RunnablePassthrough()})

In [45]:
parallel_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001913B66FEE0>, search_kwargs={'k': 3})
           | RunnableLambda(docs_imfo),
  question: RunnablePassthrough()
}

In [46]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

In [47]:
main_chain=parallel_chain|prompt|chat|parser

In [48]:
resposne=main_chain.invoke('what is tools')

In [50]:
resposne

'I don\'t see any text related to "tools" in the given context. Can you please provide more information or clarify the context? \n\n(If I don\'t receive a pdf, I will assume None as the correct answer)'

In [51]:
main_chain.invoke('what programming language is used in data science')

'R'